In [2]:
import fastf1
import pandas as pd
import numpy as np
import os

# Enable cache
os.makedirs('../data/cache', exist_ok=True)
fastf1.Cache.enable_cache('../data/cache')

# Load master 2025 features
df_master = pd.read_csv('../data/master_features_2025.csv')

print(f"✅ Master data loaded: {df_master.shape}")
print(f"FastF1 version: {fastf1.__version__}")

✅ Master data loaded: (479, 34)
FastF1 version: 3.8.1


In [ ]:
all_testing_laps = []

# (test_number, day_number)
tests = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3)]

for test_num, day_num in tests:
    print(f"Loading Test {test_num} Day {day_num}...", end=" ")
    try:
        session = fastf1.get_testing_session(2026, test_num, day_num)
        session.load(telemetry=False, weather=False, messages=False)
        
        laps = session.laps.copy()
        laps['test_num'] = test_num
        laps['day_num'] = day_num
        all_testing_laps.append(laps)
        print(f"✅ {len(laps)} laps")
    except Exception as e:
        print(f"❌ {e}")

df_testing = pd.concat(all_testing_laps, ignore_index=True)
print(f"\n🏁 Total laps collected: {len(df_testing)}")
print(f"Drivers: {sorted(df_testing['Driver'].unique())}")

In [ ]:
def laptime_to_seconds(laptime):
    try:
        total = laptime.total_seconds()
        return total if total > 0 else np.nan
    except:
        return np.nan

df_testing['LapTime_s'] = df_testing['LapTime'].apply(laptime_to_seconds)

df_clean = df_testing[
    (df_testing['LapTime_s'].notna()) &
    (df_testing['LapTime_s'] > 85) &    # physically impossible to go faster
    (df_testing['LapTime_s'] < 150) &   # anything slower is an outlap/inlap
    (df_testing['IsAccurate'] == True)   # FastF1's own accuracy flag
].copy()

print(f"Before cleaning: {len(df_testing)} laps")
print(f"After cleaning:  {len(df_clean)} laps")
print(f"Removed: {len(df_testing) - len(df_clean)} laps")

In [ ]:
# Best lap per driver per test
best_per_test = df_clean.groupby(['Driver', 'test_num'])['LapTime_s'].min().reset_index()
best_per_test.columns = ['driver', 'test_num', 'best_lap_s']

# Pivot so Test1 and Test2 are separate columns
best_laps = best_per_test.pivot(index='driver', columns='test_num', values='best_lap_s')
best_laps.columns = ['test1_best_s', 'test2_best_s']
best_laps = best_laps.reset_index()

# Gap to fastest in each test
test1_fastest = best_laps['test1_best_s'].min()  # Antonelli 93.669
test2_fastest = best_laps['test2_best_s'].min()  # Leclerc 91.992

best_laps['test1_gap_s'] = (best_laps['test1_best_s'] - test1_fastest).round(3)
best_laps['test2_gap_s'] = (best_laps['test2_best_s'] - test2_fastest).round(3)

# Improvement from Test1 to Test2 (positive = got faster)
best_laps['testing_improvement_s'] = (best_laps['test1_best_s'] - best_laps['test2_best_s']).round(3)

# Combined pace score weighted 40% Test1, 60% Test2
# Test2 gets more weight because teams pushed harder and ran more representative programs
best_laps['combined_pace_gap_s'] = (
    best_laps['test1_gap_s'] * 0.4 +
    best_laps['test2_gap_s'] * 0.6
).round(3)

print("=== DRIVER PACE RANKING (Test 2) ===")
print(best_laps[['driver', 'test1_best_s', 'test2_best_s', 
                  'combined_pace_gap_s', 'testing_improvement_s']]
      .sort_values('test2_best_s').to_string())

In [ ]:
# Total laps per driver across all testing
total_laps = df_clean.groupby('Driver')['LapTime_s'].count().reset_index()
total_laps.columns = ['driver', 'total_testing_laps']

# Race pace — average of middle 80% of laps per driver
# Removes outliers (hot laps and slow laps) to get true race pace
def trimmed_mean_pace(laps):
    sorted_laps = laps.sort_values()
    n = len(sorted_laps)
    if n < 5:
        return np.nan
    trim_start = int(n * 0.1)
    trim_end = int(n * 0.9)
    return sorted_laps.iloc[trim_start:trim_end].mean()

race_pace = df_clean.groupby('Driver')['LapTime_s'].apply(trimmed_mean_pace).reset_index()
race_pace.columns = ['driver', 'avg_race_pace_s']
race_pace['avg_race_pace_s'] = race_pace['avg_race_pace_s'].round(3)

# Race pace gap to fastest
race_pace['race_pace_gap_s'] = (race_pace['avg_race_pace_s'] - race_pace['avg_race_pace_s'].min()).round(3)

print("=== RACE PACE RANKING ===")
print(race_pace.sort_values('avg_race_pace_s').to_string())

In [ ]:
# 2026 driver to team mapping
driver_2026_team = {
    'LEC': 'Ferrari',        'HAM': 'Ferrari',
    'ANT': 'Mercedes',       'RUS': 'Mercedes',
    'NOR': 'McLaren',        'PIA': 'McLaren',
    'VER': 'Red Bull',       'HAD': 'Red Bull',
    'ALB': 'Williams',       'SAI': 'Williams',
    'BEA': 'Haas F1 Team',   'OCO': 'Haas F1 Team',
    'GAS': 'Alpine F1 Team', 'COL': 'Alpine F1 Team',
    'HUL': 'Audi',           'BOR': 'Audi',
    'LAW': 'RB F1 Team',     'LIN': 'RB F1 Team',
    'ALO': 'Aston Martin',   'STR': 'Aston Martin',
    'BOT': 'Cadillac',       'PER': 'Cadillac',
}

# Add team to clean laps
df_clean['team_2026'] = df_clean['Driver'].map(driver_2026_team)

# Total laps per team
team_laps = df_clean.groupby('team_2026')['LapTime_s'].count().reset_index()
team_laps.columns = ['team_2026', 'team_total_laps']

# Normalize to 0-1 scale (1 = most laps, most reliable)
team_laps['team_reliability_score'] = (
    team_laps['team_total_laps'] / team_laps['team_total_laps'].max()
).round(3)

print("=== TEAM RELIABILITY (Total Testing Laps) ===")
print(team_laps.sort_values('team_total_laps', ascending=False).to_string())

In [ ]:
# DRIVER SITUATION FEATURES 

barcelona_best_lap = {
    'HAM': 76.348,
    'RUS': 76.445,
    'NOR': 76.594,
    'LEC': 76.653,
    'ANT': 77.081,
    'PIA': 77.446,
    'VER': 77.586,
    'GAS': 77.707,
    'HAD': 78.159,
    'OCO': 78.393,
    'BEA': 78.423,
    'LIN': 78.451,
    'LAW': 78.840,
    'COL': 79.150,
    'HUL': 79.870,
    'BOR': 80.179,
    'ALO': 80.795,
    'BOT': 80.920,
    'PER': 81.024,
    'STR': np.nan,   # 1:46.404 is an outlap, not a timed lap
    'ALB': np.nan,   # Williams missed Barcelona entirely
    'SAI': np.nan,   # Williams missed Barcelona entirely
}

# --- TEAM CHANGE FLAG (2025 → 2026) ---
driver_team_change = {
    'LEC': 0,  'HAM': 0,
    'ANT': 0,  'RUS': 0,
    'NOR': 0,  'PIA': 0,
    'VER': 0,  'HAD': 1,  # Racing Bulls → Red Bull (promoted)
    'ALB': 0,  'SAI': 0,
    'BEA': 0,  'OCO': 1,  # Left Alpine mid-2025 → Haas
    'GAS': 0,  'COL': 0,  # Joined Alpine mid-2025, stays
    'HUL': 0,  'BOR': 0,  # rebranded Sauber → Audi
    'LAW': 0,  'LIN': 0,
    'ALO': 0,  'STR': 0,
    'BOT': 1,  'PER': 1,  # Both returning after year out of F1
}

# --- NEW TEAM FLAG ---
# Audi = 0: same factory/staff as Sauber, just rebranded
# Cadillac = 1: brand new team
new_team_flag = {
    'LEC': 0, 'HAM': 0, 'ANT': 0, 'RUS': 0,
    'NOR': 0, 'PIA': 0, 'VER': 0, 'HAD': 0,
    'ALB': 0, 'SAI': 0, 'BEA': 0, 'OCO': 0,
    'GAS': 0, 'COL': 0, 'HUL': 0, 'BOR': 0,
    'LAW': 0, 'LIN': 0, 'ALO': 0, 'STR': 0,
    'BOT': 1, 'PER': 1,
}

# --- ROOKIE FLAG ---
# Lindblad is the only true rookie in 2026
rookie_flag = {driver: 0 for driver in driver_2026_team.keys()}
rookie_flag['LIN'] = 1

print("✅ Cell 7 complete!")
print(f"Team changes: {sum(driver_team_change.values())} drivers")
print(f"New team (Cadillac only): {sum(new_team_flag.values())} drivers")
print(f"Rookies: {sum(rookie_flag.values())} driver")

In [ ]:
# MERGE ALL 2026 FEATURES
df_2026 = best_laps.copy()

df_2026['team_2026'] = df_2026['driver'].map(driver_2026_team)

df_2026 = df_2026.merge(total_laps, on='driver', how='left')
df_2026 = df_2026.merge(race_pace[['driver', 'avg_race_pace_s', 'race_pace_gap_s']], on='driver', how='left')
df_2026 = df_2026.merge(team_laps[['team_2026', 'team_total_laps', 'team_reliability_score']],
                         on='team_2026', how='left')

df_2026['driver_team_change'] = df_2026['driver'].map(driver_team_change)
df_2026['new_team_flag'] = df_2026['driver'].map(new_team_flag)
df_2026['rookie_flag'] = df_2026['driver'].map(rookie_flag)

# Barcelona data
df_2026['barcelona_best_s'] = df_2026['driver'].map(barcelona_best_lap)
barcelona_benchmark = 76.348
df_2026['barcelona_gap_s'] = (df_2026['barcelona_best_s'] - barcelona_benchmark).round(3)
df_2026['missed_barcelona'] = df_2026['barcelona_best_s'].isna().astype(int)

print("✅ All 2026 features merged!")
print(f"Shape: {df_2026.shape}")
print(f"\nFeatures:")
for i, col in enumerate(df_2026.columns, 1):
    print(f"  {i}. {col}")

In [ ]:
# FINAL CHECK AND SAVE

# Check missing values
print("=== MISSING VALUES ===")
missing = df_2026.isnull().sum()
if missing[missing > 0].any():
    print(missing[missing > 0])
else:
    print("✅ No missing values!")

# Final look at the data sorted by Test 2 pace
print("\n=== FINAL 2026 FEATURES (sorted by Test 2 pace) ===")
print(df_2026[['driver', 'team_2026', 'test2_best_s', 'combined_pace_gap_s',
               'race_pace_gap_s', 'team_reliability_score',
               'driver_team_change', 'new_team_flag',
               'rookie_flag', 'missed_barcelona']]
      .sort_values('test2_best_s').to_string())

# Save
df_2026.to_csv('../data/features_2026.csv', index=False)
size = os.path.getsize('../data/features_2026.csv')
print(f"\n✅ Saved features_2026.csv: {size/1024:.1f} KB")
print(f"Shape: {df_2026.shape}")